In [3]:
#integrating preprocessing functions together
def preprocess(text):
    # import libraries
    import spacy
    from spacy.lang.en import English
    import networkx as nx
    import matplotlib.pyplot as plt
    import pandas as pd
    import nltk
    from nltk.tokenize import word_tokenize
    from nltk.tag import pos_tag
    from nltk.corpus import stopwords
    from nltk.stem import WordNetLemmatizer
    from nltk.corpus import wordnet
    import re
    
    df1=pd.DataFrame()
    df1['senten']=[text]
    
    #cleaning text
    def clean(text):
        text = re.sub('[0-9]+.\t','',str(text))
        text = re.sub('\n ','',str(text))
        text = re.sub('\n',' ',str(text))
        text = re.sub("'s",'',str(text))
        text = re.sub("-",' ',str(text))
        text = re.sub("— ",'',str(text))
        text = re.sub('\"','',str(text))
        text = re.sub("Mr\.",'Mr',str(text))
        text = re.sub("Mrs\.",'Mrs',str(text))
        text = re.sub("[\(\[].*?[\)\]]", "", str(text))
        return text

    df1['sent']=df1['senten'].apply(clean)
    #splitting into sentence
    def sentences(text):
        # split sentences and questions
        i=0
        text = re.split('[.?]', text)
        clean_sent = []
        for sent in text:
            clean_sent.append(sent)
            df1.loc[i,'senten'] = (sent)
            i+=1
        return clean_sent

    df1['sent']=df1['sent'].apply(sentences)
    df2=pd.DataFrame(columns={'senten','token','lemma','pairs','relations'})
    df2['senten']=df1['senten']
    
    # word tokenizer
    i=0
    for sent in df2['senten']:
        token_list=[]
        tokens=word_tokenize(df2.loc[i,'senten'])
        for token in tokens:
            token_list.append(token)
        df2.loc[i,'token']=token_list
        i+=1
    # lemmatizing
    all_lemma=[]
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    tokenizer = nlp.Defaults.create_tokenizer(nlp)
    i=0
    for i in range(len(df2['token'])):
        lemma_list=[]
        tokens=[]
        tokens=tokenizer(df2.loc[i,'senten'])
        for w in tokens:
            lemma_list.append(w.lemma_)
            all_lemma.append(w.lemma_)
        df2.loc[i,'lemma']=lemma_list
        i+=i  
        
        
    #joining back tokens to sentence
    i=0
    for i in range(len(df2['token'])):
        df2.loc[i,'senten']=' '.join(df2.loc[i,'lemma'])
        i+=1
       
   #subject_object pairs
    import spacy
    from spacy import displacy
    nlp = spacy.load('en_core_web_sm')

    def get_entities(sent):
        ent1 = ""
        ent2 = ""
        prv_tok_dep = ""    # dependency tag of previous token in the sentence
        prv_tok_text = ""   # previous token in the sentence
    
        prefix = ""
        modifier = ""
  
        for tok in nlp(sent):
         ## chunk 2
         # if token is a punctuation mark then move on to the next token
            if tok.dep_ != "punct":
                # check: token is a compound word or not
                if tok.dep_ == "compound":
                    prefix = tok.text
       
            # if the previous word was also a 'compound' then add the current word to it
                    if prv_tok_dep == "compound":
                        prefix = prv_tok_text + " "+ tok.text
                    
            # check: token is a modifier or not      
                if tok.dep_.endswith("mod") == True:
                    modifier = tok.text
            
            # if the previous word was also a 'compound' then add the current word to it
                    if prv_tok_dep == "compound":
                        modifier = prv_tok_text + " "+ tok.text
      
      ## chunk 3  captures subjects 
                if tok.dep_.find("subj") == True:
                    ent1 = modifier +" "+ prefix + " "+ tok.text
                    prefix = ""
                    modifier = ""
                    prv_tok_dep = ""
                    prv_tok_text = ""      

      ## chunk 4  captures objects
                if tok.dep_.find("obj") == True:
                     ent2 = modifier +" "+ prefix +" "+ tok.text
        
      # update variables
                prv_tok_dep = tok.dep_
                prv_tok_text = tok.text

        return [ent1.strip(), ent2.strip()]
    #call for entity extraction
    def entity_extract():
        i=0
        for i in range(len(df2['senten'])):
            entity_pairs=[]
            entity_pairs.append(get_entities(df2.loc[i,'senten']))
            df2.loc[i,'pairs']=entity_pairs
            i+=1
    
    entity_extract()
    #spacy rule based matching
    from spacy.matcher import Matcher 
    from spacy.tokens import Span 

    def get_relation(sent):
        doc = nlp(sent)
      # Matcher class object 
        matcher = Matcher(nlp.vocab)

      #define the pattern 
        pattern = [{'DEP':'ROOT'}, 
                {'DEP':'prep','OP':"?"},
                {'DEP':'agent','OP':"?"},  
                {'POS':'ADJ','OP':"?"}] 
        matcher.add("Matching", None, pattern) 
        matches = matcher(doc)     #returns match_id, start and stop indexes of the matched words
        span = doc[matches[0][1]:matches[0][2]] 
        return(span.text)
    #call for relation extraction
    def relation_extract():
        relations=[]
        i=0
        for i in range(len(df2['token'])):
            relations=[]
            relations=get_relation(df2.loc[i,'senten'])
            df2.loc[i,'relations']=(relations)
            i+=1
    
    relation_extract()
    #building knowledge graph
    # extract subject
    i=0
    source=[]
    target=[]
    src=[]
    relation=[]
    for i in range(50):
        src=df2.loc[i,'pairs']
        source.append(src[0][0])
        target.append(src[0][1])
        relation.append(df2.loc[i,'relations'])
        i+=1

    kg_df = pd.DataFrame({'source':source, 'target':target, 'edge':relation})
    #plotting graph
    # create a directed-graph from a dataframe
    G=nx.from_pandas_edgelist(kg_df, "source", "target",  edge_attr=True, create_using=nx.MultiDiGraph())
    plt.figure(figsize=(8,8))
    pos = nx.spring_layout(G)
    nx.draw(G, with_labels=True, node_color='skyblue', edge_cmap=plt.cm.Blues, pos = pos)
    plt.savefig('graph_new.png')
    plt.close()

    
    return df2
    
    

In [4]:
#summary
def summary(scraped_data):
    import bs4 as bs
    import urllib.request
    import re
    import nltk
    #scraped_data=open("./UNGDC+1970-2018/Converted sessions/Session 25 - 1970/IND_25_1970.txt")
    article=scraped_data.read()
    parsed_article = bs.BeautifulSoup(article,'lxml')
    paragraphs = parsed_article.find_all('p')
    article_text = ""
    for p in paragraphs:
        article_text += p.text
    article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)
    article_text = re.sub(r'\s+', ' ', article_text)
    formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text )
    formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)
    sentence_list = nltk.sent_tokenize(article_text)
    stopwords = nltk.corpus.stopwords.words('english')
    word_frequencies = {}
    for word in nltk.word_tokenize(formatted_article_text):
        if word not in stopwords:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1
                
    # weighted freq
    maximum_frequncy = max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word] = (word_frequencies[word]/maximum_frequncy) 
    
    # Calculating Sentence Scores
    sentence_scores = {}
    for sent in sentence_list:
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies.keys():
                if len(sent.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word]
                    else:
                        sentence_scores[sent] += word_frequencies[word]

    import heapq
    summary_sentences = heapq.nlargest(7, sentence_scores, key=sentence_scores.get)

    summary = ' '.join(summary_sentences)    
                
    return summary   